In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import warnings
warnings.filterwarnings('ignore')

In [3]:
df_main = pd.read_csv('data\gemstone.csv')
df_main.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [4]:
#df = df_main.drop(labels=['id','x','y','z'], axis=1)
df = df_main.drop(labels=['id'], axis=1)
df.head()

,carat,cut,color,clarity,depth,table,x,y,z,price
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [5]:
X = df.drop(labels='price', axis=1)
y= df.drop(df.columns.difference(['price']) , axis=1)

In [6]:
X

,carat,cut,color,clarity,depth,table,x,y,z
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77
...,...,...,...,...,...,...,...,...,...
193568,0.31,Ideal,D,VVS2,61.1,56.0,4.35,4.39,2.67
193569,0.70,Premium,G,VVS2,60.3,58.0,5.75,5.77,3.47
193570,0.73,Very Good,F,SI1,63.1,57.0,5.72,5.75,3.62
193571,0.34,Very Good,D,SI1,62.9,55.0,4.45,4.49,2.81


In [7]:
y

,price
0,13619
1,13387
2,2772
3,666
4,14453
...,...
193568,1130
193569,2874
193570,3036
193571,681


In [8]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193573 entries, 0 to 193572
Data columns (total 9 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   carat    193573 non-null  float64
 1   cut      193573 non-null  object 
 2   color    193573 non-null  object 
 3   clarity  193573 non-null  object 
 4   depth    193573 non-null  float64
 5   table    193573 non-null  float64
 6   x        193573 non-null  float64
 7   y        193573 non-null  float64
 8   z        193573 non-null  float64
dtypes: float64(6), object(3)
memory usage: 13.3+ MB


In [9]:
cat_cols = X.select_dtypes(include='object').columns
num_cols=X.columns[X.dtypes != 'object']

In [10]:
cat_cols

Index(['cut', 'color', 'clarity'], dtype='object')

In [11]:
num_cols

Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')

In [12]:
X['cut'].unique()

array(['Premium', 'Very Good', 'Ideal', 'Good', 'Fair'], dtype=object)

In [13]:
X['color'].unique()

array(['F', 'J', 'G', 'E', 'D', 'H', 'I'], dtype=object)

In [14]:
X['clarity'].unique()

array(['VS2', 'SI2', 'VS1', 'SI1', 'IF', 'VVS2', 'VVS1', 'I1'],
      dtype=object)

In [15]:
cut_category = [  'Fair', 'Good','Very Good','Premium','Ideal' ]
color_category=['D','E','F','G','H','I','J']
clarity_category=['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [16]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,OrdinalEncoder
## Pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [17]:
### Numerical Pipeline 
num_pipe = Pipeline(
steps= [
        ('imputer',SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ]
)

### Categorical Pipeline 
cat_pipe = Pipeline(
steps= [
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoding', OrdinalEncoder(categories=[cut_category,color_category,clarity_category])),
    ('scaler', StandardScaler())
    ]
)


preprocessor = ColumnTransformer([
    ('numerical_pipe',num_pipe,num_cols),
    ('categorical_pipe',cat_pipe,cat_cols)
])

In [18]:
### Train-Test-Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=30 , random_state=30)

In [19]:
X_train = pd.DataFrame(preprocessor.fit_transform(X_train), columns=preprocessor.get_feature_names_out())
X_test = pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [20]:
X_train.head()

,numerical_pipe__carat,numerical_pipe__depth,numerical_pipe__table,numerical_pipe__x,numerical_pipe__y,numerical_pipe__z,categorical_pipe__cut,categorical_pipe__color,categorical_pipe__clarity
0,-0.952403,-0.666241,-0.639753,-1.059334,-1.043270,-1.094767,0.872926,-1.550242,0.016566
1,0.085068,0.073391,0.402572,0.238685,0.290313,0.269729,0.872926,-0.934118,0.682440
2,1.554818,-0.019063,1.444896,1.473606,1.451528,1.460034,-0.132979,1.530381,-0.649308
3,-0.174300,-0.851149,-0.118590,0.013334,0.063513,-0.035105,0.872926,1.530381,0.016566
4,-0.195914,0.535661,-0.639753,-0.067792,-0.045351,-0.006073,0.872926,0.298132,-0.649308


In [21]:
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [22]:
def evalute_model(true,predicted):
    mae = mean_absolute_error(true,predicted)
    mse = mean_squared_error(true,predicted)
#    rmse = np.sqrt(mean_squared_error(true,predicted))
    r2_scored = r2_score(true,predicted)
    return mae , mse ,r2_scored

In [23]:
models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'ElasticNet':ElasticNet()
}



trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    y_pred=model.predict(X_test)

    mae, rmse, r2_scored=evalute_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_scored*100)

    r2_list.append(r2_scored)
    
    print('='*35)
    print('\n')
    

LinearRegression
Model Training Performance
RMSE: 2394290.9303847654
MAE: 958.3934487339568
R2 score 90.03630934646569


Lasso
Model Training Performance
RMSE: 2392506.5738750193
MAE: 957.5666643788468
R2 score 90.04373483350786


Ridge
Model Training Performance
RMSE: 2394273.004826962
MAE: 958.4172756537796
R2 score 90.03638394254358


ElasticNet
Model Training Performance
RMSE: 5110840.163145705
MAE: 1441.5739289318742
R2 score 78.73156109852563


